## Agent-based simulation of runners in a race

In [12]:
using Agents, Plots, Random

In [7]:
@agent Runner GridAgent{2} begin
    exhaustion::Float64
    pace::Int
    viewdist::Int
end

### Step evolution logic
how many lanes ahead should runners be able to see when evaluating weights for each lane?

In [72]:
function getweights(runner::Runner, model::ABM)
    dim = spacesize(model)
    pos = runner.pos
    laneweights = zeros(dim[1])

    for i in 1:dim[1],
        j in range(pos[2]; length=min(runner.viewdist, dim[2]-pos[2]))
        laneweights[i] += (0.1
            * (length(ids_in_position((i,j), model)) - ((i,j) == pos))
            * runner.exhaustion)
    end
    
    waterpoint = getproperty(model, :waterpoint)
    if 0 < waterpoint[2] - pos[2] <= runner.viewdist
        laneweights[waterpoint[1]] -= 5
    end

    return laneweights
end

getweights (generic function with 1 method)

should we allow runners to move directly to their preferred lane in a single step? (possibly more realistic agility when taking into account that the simulation space is discrete) or make them move one lane at a time? (possibly higher performance, more continuous(?) simulation)

In [67]:
function agent_step!(runner::Runner, model::ABM)
    dim = spacesize(model)
    lane, = runner.pos
    laneweights = getweights(runner, model)
    optlane = all(p -> p == laneweights[1], laneweights) ? lane : argmin(laneweights)

    # the debate: jump to or step towards new lane
    lanedist = optlane-lane
    #delta = (lanedist,
    #    lanedist == 0 ? runner.pace : min(floor(Int, runner.pace/abs(lanedist)), 1))
    delta = (sign(lanedist), runner.pace)

    newpos = runner.pos .+ delta
    if newpos[2] > dim[2]
        remove_agent!(runner, model)
        return
    end

    move_agent!(runner, newpos, model)
end

agent_step! (generic function with 1 method)

In [68]:
function model_step!(model::ABM)
    dim = spacesize(model)
    for i in 1:sum(bitrand(2dim[1]))
        pos = (rand(1:dim[1]), 1)
        exhaustion = min(randn() + 1, 0)
        pace = sum(bitrand(5)) + 1
        viewdist = 10

        add_agent!(pos, model, exhaustion, pace, viewdist)
    end
end

model_step! (generic function with 1 method)

### Running the simulation

In [69]:
function racemodel()
    dimensions = (10, 100)
    road = GridSpace(dimensions; metric=:manhattan)

    properties = Dict(:waterpoint => (dimensions[1], dimensions[2] / 2))
    model = ABM(Runner, road; properties)

    for i in 1:sum(bitrand(2dimensions[1]))
        pos = (rand(1:dimensions[1]), 1)
        exhaustion = min(randn() + 1, 0)
        pace = sum(bitrand(5)) + 1
        viewdist = 10

        add_agent!(pos, model, exhaustion, pace, viewdist)
    end

    return model
end

racemodel (generic function with 1 method)

In [73]:
model = racemodel()

StandardABM with 11 agents of type Runner
 space: GridSpace with size (10, 100), metric=manhattan, periodic=true
 scheduler: fastest
 properties: waterpoint

### Visualisation

In [ ]:
@gif for i in 1:50
    step!(model, agent_step!, model_step!, 1)

    agentcounts = map(p -> length(ids_in_position(p, model)), positions(model))
    heatmap(agentcounts, aspect_ratio = :equal)
end